In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import random 
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as Func
import torch.optim as optim
torch.manual_seed(1)
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import scipy.io

In [5]:

def load_data():

    path_feat = 'UCF101_release/vgg16_relu6/'
    path_label = 'UCF101_release/annos/videos_labels_subsets.txt'
    fp = open(path_label,'r')
    samples = os.listdir(path_feat)
    train_data = []
    train_labels = []
    test_data= []
    test_labels = []
    for line in fp.readlines():
        parts = line.strip().split('\t')
        if(int(parts[1])< 16 and not(int(parts[1])==1)):
            mat_in = scipy.io.loadmat(path_feat+parts[0]+'.mat') 
            feat_in = mat_in['Feature']
            if(parts[2]=='1'):
                train_data.append(feat_in)
                train_labels.append([int(parts[1])-1])
            if(parts[2]=='2'):
                test_data.append(feat_in)
                test_labels.append([int(parts[1])-1])
    train_data = np.array(train_data)
    train_labels = np.array(train_labels)
    test_data = np.array(test_data)
    test_labels = np.array(test_labels)
    bundle = list(zip(train_data, train_labels))
    random.shuffle(bundle)
    train_data, train_labels = zip(*bundle)
    return train_data, train_labels, test_data, test_labels


In [ ]:
train_data, train_labels, test_data, test_labels = load_data()

In [ ]:
class LSTMTagger(nn.Module):

    def __init__(self, feat_dim, hidden_dim1,hidden_dim2, cls_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim1 = hidden_dim1   # dimension of first hidden lstm layer
        self.hidden_dim2 = hidden_dim2   # dimension of second hidden lstm layer
        self.bn = nn.BatchNorm1d(25)     
        self.lstm1 = nn.LSTM(feat_dim, hidden_dim1)
        self.lstm2 = nn.LSTM(hidden_dim1,hidden_dim2)
        self.lin = nn.Linear(hidden_dim2, cls_size)
        self.hidden1 = self.init_hidden1()
        self.hidden2 = self.init_hidden2()
        self.dp = nn.Dropout(p=0.2, inplace=False)


    def init_hidden1(self):
        return (torch.zeros(1, 1, self.hidden_dim1),
                torch.zeros(1, 1, self.hidden_dim1))
    def init_hidden2(self):
        return (torch.zeros(1, 1, self.hidden_dim2),
                torch.zeros(1, 1, self.hidden_dim2))

    def forward(self, data):
        lstm_out1, self.hidden1 = self.lstm1(data.view(25,1,-1), self.hidden1)
        lstm_out2, self.hidden2 = self.lstm2(lstm_out1.view(25,1,-1), self.hidden2)
        lin_out = self.lin(lstm_out2.view(25,-1))
        cls_scores = F.log_softmax(lin_out, dim=1)
        return cls_scores[-2:-1,:]

In [ ]:
feat_dim = 4096
hidden_dim1 = 512
hidden_dim2 = 124
cls_size = 15

model = LSTMTagger(feat_dim, hidden_dim1,hidden_dim2, cls_size)
loss_function = nn.NLLLoss()
#loss_function = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.05)



In [ ]:
i = 0
for epoch in range(6):  
    for data, labels in zip(train_data,train_labels):

        #Step 1 : clear gardients and hidden states
        model.zero_grad()
        model.hidden1 = model.init_hidden1()
        model.hidden2 = model.init_hidden2()

        # Step 2. Get our inputs ready for the network, that is, turn them into tensors
        data = torch.from_numpy(np.array(data))
        data = data.type(torch.FloatTensor)
        labels = torch.from_numpy(np.array(labels))
        labels = labels.type(torch.LongTensor)

        # Step 3. Run our forward pass.
        cls_scores = model(data)

        # Step 4. Compute the loss, gradients, and update the parameters 
        loss = loss_function(cls_scores, labels)
       # i+= 1
        #if(i%10 == 0):
         #   print(i,loss[0])
        
        loss.backward()
        optimizer.step()

In [1]:
  # again, normally you would NOT do 300 epochs, it is toy data
running_corrects = 0
for data, labels in zip(test_data,test_labels):
    
    with torch.no_grad():
        model.hidden1 = model.init_hidden1()
        model.hidden2 = model.init_hidden2()

        data = torch.from_numpy(np.array(data))
        data = data.type(torch.FloatTensor)
        cls_scores = model(data)
        labels = torch.from_numpy(np.array(labels))
        labels = labels.type(torch.LongTensor)
        _, preds = torch.max(cls_scores, 1)
        running_corrects += torch.sum(preds == labels.data)
        #print(preds,labels)
accuracy = running_corrects*100/test_data.shape[0]


print("accuracy of my model =",int(accuracy))
#print(running_corrects)

('accuracy of my model =', 94)


In [6]:
# Write your codes here
train_data, train_labels, test_data, test_labels = load_data()
train_data = np.array(train_data)
from sklearn.svm import LinearSVC
print(train_data.shape)
train_svm = np.reshape(train_data,(train_data.shape[0],-1))
train_labels = np.reshape(train_labels,(-1))
test_labels = np.reshape(test_labels,(-1))

test_svm = np.reshape(test_data,(test_data.shape[0],-1))
clf3 = LinearSVC(random_state=0, tol=1e-5, max_iter = 10000, C= 0.2)
clf3.fit(train_svm, train_labels)
predictions3 = clf3.predict(test_svm)
predictions3 = np.array(predictions3)
test_labels = np.array(test_labels)    
accuracy = sum(np.array(predictions3) == test_labels) / float(test_data.shape[0])

print "The accuracy of my svm model is {:.2f}%".format(accuracy*100)


#pred3, label3 = # train_and_test(...

(1341, 25, 4096)
The accuracy of my dummy model is 95.23%
